## VarQRTE Error

In [ ]:
from jax import grad, jit
import jax.numpy as jnp

import os
import warnings

import numpy as np
import scipy as sp
from scipy.linalg import expm

from qiskit.circuit.library import EfficientSU2, RealAmplitudes

from qiskit.opflow import StateFn, MatrixOp, CircuitOp, X, I, PauliOp, Z, Y, SummedOp
from qiskit.opflow.gradients import NaturalGradient
from qiskit.opflow.evolutions.varqtes.varqrte import VarQRTE

np.random.seed = 2

In [7]:
# Set Hamiltonian
# Hamiltonian = SummedOp([(Z ^ X), 0.3 *(Y ^ Y), (Y ^ I)]).reduce()
Hamiltonian = SummedOp([(Z ^ X), 0.8 *(Y ^ Y)]).reduce() # works
# Hamiltonian = SummedOp([(Z ^ X), (X ^ Z), 3 * (Z ^ Z)]).reduce()
Hamiltonian = observable = SummedOp([0.25 * (I ^ X), 0.25 * (X ^ I), (Z ^ Z)]).reduce()
# Hamiltonian = SummedOp([(Z ^ X), 3. * (Y ^ Y), (Z ^ X), (I ^ Z), (Z ^ I)]).reduce() #works
# Hamiltonian = (Z ^ X) # works
# Hamiltonian = (Y ^ I)
# Set time and time_steps 
time = 1
time_steps = 20

In [8]:
T = Hamiltonian **2
print(T.to_pauli_op().reduce())

1.125 * II
+ 0.125 * XX


In [9]:
# Helper Function computing the inner product
def inner_prod(x, y):
    return np.matmul(np.conj(np.transpose(x)), y)

In [10]:
# Set Ansatz and initial parameters
ansatz = EfficientSU2(2, reps=1, entanglement='linear')
parameters = ansatz.ordered_parameters
init_param_values = np.zeros(len(ansatz.ordered_parameters))
for i in range(ansatz.num_qubits):
    init_param_values[-(ansatz.num_qubits + i + 1)] = np.pi / 2
    
    

### Analytic Calculations

In [11]:
def analytic_error(state, H, et_ket=None):
    if et_ket is None:
        et_ket = np.zeros(len(state))
        
    dt_state = -1j*(np.dot(H, state) + et_ket)

    # <dt|dt>
    et = inner_prod(dt_state, dt_state) 
    #<H^2>
    et += inner_prod(state, np.matmul(np.matmul(H, H), state))
    # 2Im<dt|H|state>
    et -= 2 * np.imag(inner_prod(dt_state, np.matmul(H, state)))
    
    print('Gradient error', np.round(np.sqrt(et), 5))
    return np.sqrt(et), dt_state

In [12]:
num_qubits = Hamiltonian.num_qubits
H = Hamiltonian.to_matrix(massive=True)

# Propagation Unitary
evolution_op = lambda t: expm(-1 * H * t)
# Initial State
init_state = StateFn(ansatz).assign_parameters(dict(zip(parameters, init_param_values))).eval().primitive

evolution_op = lambda t: expm(-1j * H * t)
target_state = lambda t: np.dot(evolution_op(t), init_state)
dt_target_state = lambda t: np.dot(-1j * H, target_state(t))


error = 0
prepared_state = target_state(0)
for j in range(0, time_steps):
    et_ket = np.ones(2 ** num_qubits) / np.sqrt(2 ** num_qubits) * j * 1e-4

    et, dt_prepared_state = analytic_error(prepared_state, H, et_ket=et_ket)
    # Euler State propagation
    prepared_state += time/time_steps * dt_prepared_state
    # Compute Error
#     error += time/time_steps * et
    error += time/time_steps * np.sqrt(np.linalg.norm(et_ket))

print('error bound for time', np.round(time/time_steps*(j+1), 3), np.round(error, 3))

# sqrt of the l2-norm of the distance between the perturbed, prepared state and the exat, target state
print('True error ', np.sqrt(np.linalg.norm(prepared_state - target_state(time))))


Gradient error 0j
Gradient error (0.0001+0j)
Gradient error (0.0002+0j)
Gradient error (0.0003+0j)
Gradient error (0.0004+0j)
Gradient error (0.0005-0j)
Gradient error (0.0006+0j)
Gradient error (0.0007+0j)
Gradient error (0.0008+0j)
Gradient error (0.0009+0j)
Gradient error (0.001+0j)
Gradient error (0.0011+0j)
Gradient error (0.0012-0j)
Gradient error (0.0013-0j)
Gradient error (0.0014+0j)
Gradient error (0.0015+0j)
Gradient error (0.0016+0j)
Gradient error (0.0017+0j)
Gradient error (0.0018+0j)
Gradient error (0.0019-0j)
error bound for time 1.0 0.029
True error  0.17890955189857174


### Numpy Calculations

In [13]:
def dt_params(a, c, regularization=None):
    a = np.real(a)
    c = np.real(c)
    if regularization:
        # If a regularization method is chosen then use a regularized solver to
        # construct the natural gradient.
        nat_grad = NaturalGradient._regularized_sle_solver(
            a, c, regularization=regularization)
    else:
        nat_grad = np.linalg.lstsq(a, c, rcond=None)[0]
        if np.linalg.norm(nat_grad) < 1e-8:
            nat_grad = NaturalGradient._regularized_sle_solver(a,
                                                               c,
                                                               regularization='perturb_diag')
            warnings.warn(r'Norm of the natural gradient smaller than $1e^{-8}$ use '
                          r' `perturb_diag` regularization.')
        if np.linalg.norm(nat_grad) > 1e-4:
            nat_grad = NaturalGradient._regularized_sle_solver(a,
                                                               c,
                                                               regularization='ridge')
            warnings.warn(r'Norm of the natural gradient bigger than $1e^{3}$ use '
                          r' `ridge` regularization.')
#         try:
#             # Try to solve the system of linear equations Ax = C.
#             nat_grad = np.linalg.solve(a, c)
#         except np.linalg.LinAlgError:  # singular matrix
#             print('Singular matrix lstsq solver required')
#             nat_grad, resids, _, _ = np.linalg.lstsq(a, c)
#             print('Residuals', resids)
    return np.real(nat_grad)

In [14]:
def numpy_error(a, c, dt_weights, H, state):

    dtdt = inner_prod(dt_weights, np.matmul(a, dt_weights))
    et = dtdt
    print('dtdt', dtdt)
    
    h_squared = inner_prod(state, np.matmul(np.matmul(H, H), state))
    
    if h_squared < dtdt:
        print('Eq. 8 violated')
    et = np.add(et, h_squared)
    
    print('H^2', h_squared)

    
    dt = 2*inner_prod(dt_weights, c)
    et -= dt
    print('2Im<dt|H|>', dt)
    
    print('Grad error', np.round(np.sqrt(et), 6))
    print('|adtw-c|', np.linalg.norm(np.matmul(a, dt_weights)-c))
    return np.sqrt(et)

In [15]:
print('Warning: Numpy Calculations are only compatible with SU2 depth 1 - else hard-coded changes needed.')

def ry(theta):
    return jnp.array([[jnp.cos(theta/2.), -1*jnp.sin(theta/2)], [jnp.sin(theta/2),
                                                                 jnp.cos(theta/2)]])

def rz(theta):
    return jnp.array([[jnp.exp(-1j * theta / 2.), 0], [0, jnp.exp(1j * theta / 2.)]])


def ryry(alpha, beta):
    return jnp.kron(ry(alpha), ry(beta))

def rzrz(alpha, beta):
    return jnp.kron(rz(alpha), rz(beta))

cx = jnp.array([[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]])
i = jnp.eye(2)
y = jnp.array([[0, -1j], [1j, 0]])
z = jnp.array([[1, 0], [0, -1]])
iy = -0.5j * jnp.kron(i, y)
yi = -0.5j * jnp.kron(y, i)
iz = -0.5j * jnp.kron(i, z)
zi = -0.5j * jnp.kron(z, i)

init = jnp.array([1, 0, 0, 0])

def state_fn(params):
    vec = jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]), jnp.dot(ryry(params[1],
                                                                                params[0]),
                                                                           init))))
    return vec

def state0(params):
    vec = jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]), jnp.dot(ryry(params[1],
                                                                                params[0]),
                                                                           init))))
    return vec[0]

def state1(params):
    vec = jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]), jnp.dot(ryry(params[1],
                                                                                params[0]),
                                                                           init))))
    return vec[1]

def state2(params):
    vec = jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]), jnp.dot(ryry(params[1],
                                                                                params[0]),
                                                                           init))))
    return vec[2]

def state3(params):
    vec = jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]), jnp.dot(ryry(params[1],
                                                                                params[0]),
                                                                           init))))
    return vec[3]


def A(vec, gradient):
    vec = np.reshape(vec, (len(vec), 1))
    a = np.real((inner_prod(gradient, gradient)))
#     print('a carrying part', np.round(a, 3))
    a = np.subtract(a, np.real(np.matmul(inner_prod(gradient, vec),
                    np.transpose(np.conj(inner_prod(gradient, vec))))))
#     print('a phase fix', np.round(np.real(np.matmul(inner_prod(gradient, vec),
#                     np.transpose(np.conj(inner_prod(gradient, vec))))), 3))
    return a

def C(vec, gradient, h):
    vec = np.reshape(vec, (len(vec), 1))
    c = np.imag(inner_prod(gradient, np.matmul(h, vec)))
    print('carrying part c', np.round(c, 3))
    c = np.add(c, np.real(1j * inner_prod(gradient, vec) * inner_prod(vec, np.matmul(h, vec))))
    print('phase fix c', np.round(np.real(1j * inner_prod(gradient, vec) * inner_prod(vec, np.matmul(h, vec))), 3))
#     print('c', c)
    return c

def grad0(params):
    try:
        return grad(state0)(params)
    except Exception:
        return grad(state0, holomorphic=True)(jnp.complex64(params))
def grad1(params):
    try:
        return grad(state1)(params)
    except Exception:
        return grad(state1, holomorphic=True)(jnp.complex64(params))
def grad2(params):
    try:
        return grad(state2)(params)
    except Exception:
        return grad(state2, holomorphic=True)(jnp.complex64(params))
def grad3(params):
    try:
        return grad(state3)(params)
    except Exception:
        return grad(state3, holomorphic=True)(jnp.complex64(params))


/Users/ouf/anaconda3/envs/QiskitDevenv/lib/python3.7/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [16]:
init_param_values = np.zeros(len(ansatz.ordered_parameters))
for i in range(ansatz.num_qubits):
    init_param_values[-(ansatz.num_qubits + i + 1)] = np.pi / 2
params = init_param_values
error = 0

for j in range(time_steps):

    # dim vec x num_params
    gradient = [grad0(params), grad1(params), grad2(params), grad3(params)]
    gradient = np.array([[complex(item) for item in g] for g in gradient]).astype(
        np.complex)

    state = state_fn(params)
    state = np.array([complex(s) for s in state]).astype(np.complex)

    metric = A(state, gradient)
    c_grad = C(state, gradient, H)
    print('grad res', np.round(c_grad*2, 3))
    print('metric', np.round(metric, 3))

    dt_weights = dt_params(metric, c_grad, 'ridge')
    print('dt_weights', dt_weights)

    et = numpy_error(metric, c_grad, dt_weights, H, state)

    error += time/time_steps * et

    params += time/time_steps * np.reshape(dt_weights, np.shape(params))
    print('params', params)
    print('error bound for time', np.round(time/time_steps*(j+1), 3), np.round(error, 3))

# sqrt of the l2-norm of the distance between the perturbed, prepared state and the exat, target state
print('True error ', np.sqrt(np.linalg.norm(state_fn(params) - target_state(time))))
print(state_fn(params))
print(target_state(time))

carrying part c [[0.  ]
 [0.  ]
 [0.25]
 [0.25]
 [0.  ]
 [0.  ]
 [0.  ]
 [0.  ]]
phase fix c [[ 0.  ]
 [ 0.  ]
 [-0.25]
 [-0.25]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]]
grad res [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
metric [[0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.25]]
dt_weights [[-1.82197561e-08]
 [-1.69183463e-08]
 [ 1.07046804e-14]
 [ 1.07046804e-14]
 [-1.82197561e-08]
 [-1.69183463e-08]
 [-1.82197561e-08]
 [-1.82197561e-08]]
dtdt [[6.18189879e-16]]
H^2 (1.2499998509883925+0j)
2Im<dt|H|> [[1.27609716e-21]]
Grad error [[1.118034+0.j]]
|adtw-c| 4.4748869865304256e-08
params [-9.10987805e-10 -8.45917313e-10  5.35234022e-16  5.35234022e-16
  1.57079633e+00  1.57079633e+00 -9.10987805e-

carrying part c [[-0.  ]
 [ 0.  ]
 [ 0.25]
 [ 0.25]
 [ 0.  ]
 [ 0.  ]
 [-0.  ]
 [-0.  ]]
phase fix c [[ 0.  ]
 [-0.  ]
 [-0.25]
 [-0.25]
 [-0.  ]
 [-0.  ]
 [ 0.  ]
 [ 0.  ]]
grad res [[-0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-0.]
 [-0.]]
metric [[ 0.25  0.   -0.   -0.    0.    0.   -0.    0.  ]
 [ 0.    0.25  0.    0.    0.    0.25 -0.    0.  ]
 [-0.    0.    0.    0.    0.    0.   -0.   -0.  ]
 [-0.    0.    0.    0.    0.    0.   -0.   -0.  ]
 [ 0.    0.    0.    0.    0.25  0.   -0.   -0.  ]
 [ 0.    0.25  0.    0.    0.    0.25 -0.   -0.  ]
 [-0.   -0.   -0.   -0.   -0.   -0.    0.25  0.  ]
 [ 0.    0.   -0.   -0.   -0.   -0.    0.    0.25]]
dt_weights [[-1.85528830e-08]
 [-1.59158895e-08]
 [ 1.05756183e-14]
 [ 1.05756183e-14]
 [-1.71401873e-08]
 [-1.59158895e-08]
 [-1.85528797e-08]
 [-1.85528797e-08]]
dtdt [[5.84919012e-16]]
H^2 (1.2499998509883925+0j)
2Im<dt|H|> [[-8.9935683e-17]]
Grad error [[1.118034+0.j]]
|adtw-c| 4.500603806407596e-08
params [-8.27364575e-09 -7.38829901e-09

carrying part c [[-0.  ]
 [ 0.  ]
 [ 0.25]
 [ 0.25]
 [ 0.  ]
 [ 0.  ]
 [-0.  ]
 [-0.  ]]
phase fix c [[ 0.  ]
 [-0.  ]
 [-0.25]
 [-0.25]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]]
grad res [[-0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-0.]
 [-0.]]
metric [[ 0.25 -0.   -0.   -0.    0.    0.   -0.    0.  ]
 [-0.    0.25  0.    0.    0.    0.25  0.    0.  ]
 [-0.    0.    0.    0.    0.    0.   -0.   -0.  ]
 [-0.    0.    0.    0.    0.    0.   -0.   -0.  ]
 [ 0.    0.    0.    0.    0.25  0.   -0.    0.  ]
 [ 0.    0.25  0.    0.    0.    0.25  0.   -0.  ]
 [-0.    0.   -0.   -0.   -0.    0.    0.25  0.  ]
 [ 0.    0.   -0.   -0.    0.   -0.    0.    0.25]]
dt_weights [[-1.88921005e-08]
 [-1.48951042e-08]
 [ 1.04441965e-14]
 [ 1.04441965e-14]
 [-1.60408802e-08]
 [-1.48951042e-08]
 [-1.88920939e-08]
 [-1.88920939e-08]]
dtdt [[5.53874994e-16]]
H^2 (1.2499998509883925-8.271806125530277e-25j)
2Im<dt|H|> [[-1.69875097e-16]]
Grad error [[1.118034-0.j]]
|adtw-c| 4.5284314141260616e-08
params [-1.577092

### Variational Calculations

In [17]:
import os
from qiskit import Aer
init_param_values = np.zeros(len(ansatz.ordered_parameters))
for i in range(ansatz.num_qubits):
    init_param_values[-(ansatz.num_qubits + i + 1)] = np.pi / 2
op = ~StateFn(Hamiltonian)@StateFn(ansatz)
backend = Aer.get_backend('statevector_simulator')
# op = ~StateFn(ansatz)@Hamiltonian@StateFn(ansatz)
op = time * op
approx_time_evolved_state = VarQRTE(parameters=parameters,
                                    grad_method='lin_comb',
                                    init_parameter_values=init_param_values, num_time_steps=time_steps,
                                    regularization='ridge', backend=backend,
                                    error_based_ode=False,
                                    snapshot_dir=os.path.join('dummy')).convert(op)

nat grad result [ 3.90565881e-17 -9.50380059e-18 -5.65039324e-27 -5.65039617e-27
 -6.82576506e-18 -9.50380059e-18  3.11541146e-18  2.40952910e-18]
returned et 1.25
after try except 1.118033988749895
nat grad result [ 3.90565881e-17 -9.50380059e-18 -5.65039324e-27 -5.65039617e-27
 -6.82576506e-18 -9.50380059e-18  3.11541146e-18  2.40952910e-18]
nat grad result [ 3.90565881e-17 -9.50380059e-18 -5.65039324e-27 -5.65039617e-27
 -6.82576506e-18 -9.50380059e-18  3.11541146e-18  2.40952910e-18]
returned et 1.25
after try except 1.118033988749895
ode time 0.05
ode parameters [1.952829406508075e-18, -4.751900294910835e-19, -2.8251966216768343e-28, -2.8251980826351234e-28, 1.5707963267948966, 1.5707963267948966, 1.5577057304629333e-19, 1.2047645520416352e-19]
ode step size 0.05
nat grad result [ 3.90565881e-17 -9.50380059e-18 -5.65039324e-27 -5.65039617e-27
 -6.82576506e-18 -9.50380059e-18  3.11541146e-18  2.40952910e-18]
nat grad result [ 3.90565881e-17 -9.50380059e-18 -5.65039324e-27 -5.650396

nat grad result [ 3.49522844e-17 -9.18111688e-18 -5.61971989e-27 -5.61972025e-27
 -3.34183345e-18 -9.18111688e-18  3.18757835e-18  2.48169599e-18]
nat grad result [ 3.49522844e-17 -9.18111688e-18 -5.61971989e-27 -5.61972025e-27
 -3.34183345e-18 -9.18111688e-18  3.18757835e-18  2.48169599e-18]
returned et 1.25
after try except 1.118033988749895
ode time 0.7500000000000001
ode parameters [2.7659965989937183e-17, -6.823058108973439e-18, -4.11159660344533e-27, -4.111597972668876e-27, 1.5707963267948966, 1.5707963267948966, 2.633495263380229e-18, 2.1040834957482826e-18]
ode step size 0.05
nat grad result [ 3.49522844e-17 -9.18111688e-18 -5.61971989e-27 -5.61972025e-27
 -3.34183345e-18 -9.18111688e-18  3.18757835e-18  2.48169599e-18]
nat grad result [ 3.49522844e-17 -9.18111688e-18 -5.61971989e-27 -5.61972025e-27
 -3.34183345e-18 -9.18111688e-18  3.18757835e-18  2.48169599e-18]
returned et 1.25
after try except 1.118033988749895
ode time 0.8000000000000002
ode parameters [2.940758021019682e-

In [18]:
test = ~StateFn(ansatz)@Hamiltonian@StateFn(ansatz)
print(test.assign_parameters(dict(zip(ansatz.ordered_parameters, init_param_values))).eval())

(0.5+0j)


In [19]:
print(test)

ComposedOp([
  CircuitMeasurement(
       ┌───────────────┐┌───────────────┐     ┌───────────────┐┌───────────────┐
  q_0: ┤ RZ(-1.0*θ[6]) ├┤ RY(-1.0*θ[4]) ├──■──┤ RZ(-1.0*θ[2]) ├┤ RY(-1.0*θ[0]) ├
       ├───────────────┤├───────────────┤┌─┴─┐├───────────────┤├───────────────┤
  q_1: ┤ RZ(-1.0*θ[7]) ├┤ RY(-1.0*θ[5]) ├┤ X ├┤ RZ(-1.0*θ[3]) ├┤ RY(-1.0*θ[1]) ├
       └───────────────┘└───────────────┘└───┘└───────────────┘└───────────────┘
  ),
  1.0 * ZZ
  + 0.25 * XI
  + 0.25 * IX,
  CircuitStateFn(
       ┌──────────┐┌──────────┐     ┌──────────┐┌──────────┐
  q_0: ┤ RY(θ[0]) ├┤ RZ(θ[2]) ├──■──┤ RY(θ[4]) ├┤ RZ(θ[6]) ├
       ├──────────┤├──────────┤┌─┴─┐├──────────┤├──────────┤
  q_1: ┤ RY(θ[1]) ├┤ RZ(θ[3]) ├┤ X ├┤ RY(θ[5]) ├┤ RZ(θ[7]) ├
       └──────────┘└──────────┘└───┘└──────────┘└──────────┘
  )
])


In [20]:
Hamiltonian

PauliSumOp(SparsePauliOp([[False, False,  True,  True],
               [False,  True, False, False],
               [ True, False, False, False]],
              coeffs=[1.  +0.j, 0.25+0.j, 0.25+0.j]), coeff=1.0)